DLC_JAABA_pipeline Step 2-1
convert pose positions to fly and inter-fly features

Related to Cell manuscript "Male-Male Interactions Shape Mate Selection in Drosophila" 
by Tom Hindmarsh Sten, Rufei Li, Florian Hollunder, Shade Eleazer, and Vanessa Ruta

Description: 
This script was provided along with other scripts to show how we converted DeepLabCut pose tracking output to JAABA-classified behavioral epochs.

Input: 
.h5 files from DeepLabCut pose tracking (make sure there is no identity swaps between M and F)

Output: 
.csv files for individual flies in each assay including useful fly and inter-fly features

Dependencies: 
Python packages see below, no required versions

Last updated on 2024-12-09 

In [1]:
import pandas as pd
import numpy as np
from numpy import pi
from scipy import signal
import os
import plotly.express as px
import plotly.io as pio

from plotly.subplots import make_subplots
pio.renderers.default = "plotly_mimetype+notebook"

In [2]:
def getBodypartDistance(dataframe, partname1, partname2):
    # retrieves the pixel distance between two bodyparts

    bpt1 = dataframe.xs(partname1, level='bodyparts', axis=1).to_numpy()
    bpt2 = dataframe.xs(partname2, level='bodyparts', axis=1).to_numpy()

    bptDistance = np.sqrt(np.sum(np.square(bpt1[:, [0, 1]] - bpt2[:, [0, 1]]), axis=1))
    return bptDistance


def getAnimalCentroid(dataframe):
    scorer = dataframe.columns.get_level_values(0).unique()[0]
    bptNames = dataframe.columns.get_level_values(1).unique()
    aniXpos, aniYpos = np.zeros((nFrames, len(bptNames))), np.zeros((nFrames, len(bptNames)))

    for i, nm in enumerate(bptNames):
        aniXpos[:, i] = dataframe[scorer][nm]['x']
        aniYpos[:, i] = dataframe[scorer][nm]['y']

    xCenter, yCenter = np.nanmean(aniXpos, axis=1), np.nanmean(aniYpos, axis=1)
    centroid = np.column_stack((xCenter, yCenter))

    return centroid


def getBodypartAngle(dataframe, partName1, partName2):
    # retrieves the angle between two bodyparts

    bpt1 = dataframe.xs(partName1, level='bodyparts', axis=1).to_numpy()
    bpt2 = dataframe.xs(partName2, level='bodyparts', axis=1).to_numpy()

    angle = np.arctan2(bpt2[:, 1] - bpt1[:, 1], bpt2[:, 0] - bpt1[:, 0])
    return angle


def circularDistance(ang1, ang2):
    # efficiently computes the circular distance between two angles

    circdist = np.angle(np.exp(1j * ang1) / np.exp(1j * ang2))

    return circdist


def wrapTo2pi(ang):
    # wraps a set of values to fit between zero and 2Pi

    positiveValues = (ang > 0)
    wrappedAng = ang % (2 * pi)
    wrappedAng[ang == 0 & positiveValues] = 2 * pi

    return wrappedAng


def getRelativeOrientations(ani1, ani2):

    normPos = ani2[['XCenter', 'YCenter']] - ani1[['XCenter', 'YCenter']]
    absoluteAngle = np.arctan2(normPos['YCenter'], normPos['XCenter'])
    fA = circularDistance(absoluteAngle, ani2['heading'])
    aBetween = circularDistance(ani1['heading'],ani2['heading'])

    return fA, aBetween


def removeJumps(dataframe, maxJumpLength):
    # removes large jumps in the x/y position of bodyparts, usually resulting from swaps between animals

    # get all column names
    scorer = dataframe.columns.get_level_values(0)[0]
    bps = list(dataframe.columns.levels[1])
    params = list(dataframe.columns.levels[2])
    dataframeMod = dataframe.copy()

    for i, partName in enumerate(bps):

        xDiff = pd.Series(np.diff(dataframe[scorer][partName]['x']))
        yDiff = pd.Series(np.diff(dataframe[scorer][partName]['y']))

        xJumpsPositive = signal.find_peaks(xDiff.interpolate(), threshold=200)
        xJumpsNegative = signal.find_peaks(xDiff.interpolate() * -1, threshold=200)
        yJumpsPositive = signal.find_peaks(yDiff.interpolate(), threshold=200)
        yJumpsNegative = signal.find_peaks(yDiff.interpolate() * -1, threshold=200)

        toKill = np.zeros((len(yDiff),), dtype=bool)

        for j in range(len(xJumpsPositive[0])):
            if np.any((xJumpsNegative[0] > xJumpsPositive[0][j]) & (
                    xJumpsNegative[0] < xJumpsPositive[0][j] + maxJumpLength)):
                endIdx = np.where((xJumpsNegative[0] > xJumpsPositive[0][j]) & (
                        xJumpsNegative[0] < xJumpsPositive[0][j] + maxJumpLength))
                toKill[xJumpsPositive[0][j]:xJumpsNegative[0][endIdx[0][0]]] = True
            else:
                toKill[xJumpsPositive[0][j]] = True

        for j in range(len(xJumpsNegative[0])):

            if np.any((xJumpsPositive[0] > xJumpsNegative[0][j]) & (
                    xJumpsPositive[0] < xJumpsNegative[0][j] + maxJumpLength)):
                endIdx = np.where((xJumpsPositive[0] > xJumpsNegative[0][j]) & (
                        xJumpsPositive[0] < xJumpsNegative[0][j] + maxJumpLength))
                toKill[xJumpsNegative[0][j]:xJumpsPositive[0][endIdx[0][0]]] = True
            else:
                toKill[xJumpsNegative[0][j]] = True

        for j in range(len(yJumpsPositive[0])):
            if np.any((yJumpsNegative[0] > yJumpsPositive[0][j]) & (
                    yJumpsNegative[0] < yJumpsPositive[0][j] + maxJumpLength)):
                endIdx = np.where((yJumpsNegative[0] > yJumpsPositive[0][j]) & (
                        yJumpsNegative[0] < yJumpsPositive[0][j] + maxJumpLength))
                toKill[yJumpsPositive[0][j]:yJumpsNegative[0][endIdx[0][0]]] = True
            else:
                toKill[yJumpsPositive[0][j]] = True

        for j in range(len(yJumpsNegative[0])):
            if np.any((yJumpsPositive[0] > yJumpsNegative[0][j]) & (
                    yJumpsPositive[0] < yJumpsNegative[0][j] + maxJumpLength)):
                endIdx = np.where((yJumpsPositive[0] > yJumpsNegative[0][j]) & (
                        yJumpsPositive[0] < yJumpsNegative[0][j] + maxJumpLength))
                toKill[yJumpsNegative[0][j]:yJumpsPositive[0][endIdx[0][0]]] = True
            else:
                toKill[yJumpsNegative[0][j]] = True

        toKill = np.insert(toKill, 1, False)

        dataframeMod.loc[toKill, (scorer, partName, params)] = np.nan

    return dataframeMod


def butterLowpassFilter(data, cutoff, fs, order):
    # implements a butterworth lowpass filter to a signal (data) sampled at fs, with a set cutoff.

    nyq = fs / 2
    adjustedCutoff = cutoff / nyq
    b, a = signal.butter(order, adjustedCutoff, btype='low', analog=False)
    filteredSignal = signal.filtfilt(b, a, data)

    return filteredSignal


def getContinousNumbers(nums):
    # returns a list of continous numbers found in an array

    nums = sorted(set(nums))
    gaps = [[s, e] for s, e in zip(nums, nums[1:]) if s + 1 < e]
    edges = iter(nums[:1] + sum(gaps, []) + nums[-1:])
    return list(zip(edges, edges))

In [3]:
folder_pathname = '/Users/rufeili/Documents/Test/Example_MMF'
os.chdir(folder_pathname)
folders = os.listdir(folder_pathname)
for folder in folders:
    if folder.startswith('.'):
        continue

    pathname = os.path.join(folder_pathname, folder)
    os.chdir(pathname)
    files = os.listdir(pathname)

    for file in files:
        if file.endswith('.h5'):
            filename = file

            tracks = pd.read_hdf(os.path.join(pathname, filename))
            scorer = tracks.columns.get_level_values(0)[0]
            sampleRate = 60  # Hz
            maxJump = 6
            tstamp = np.linspace(0, len(tracks) * 1 / sampleRate, len(tracks))
            nFrames = len(tracks)

            male1Name = 'ind1'
            male2Name = 'ind2'
            femaleName = 'ind3' 
            
            # use abdomen length to get the actual ind number for female
            femalePositions = tracks.xs(femaleName, level='individuals', axis=1)
            female_abdomenlength = getBodypartDistance(femalePositions, 'abdomenTop', 'genitalia')
            male1Positions = tracks.xs(male1Name, level='individuals', axis=1)
            male1_abdomenlength = getBodypartDistance(male1Positions, 'abdomenTop', 'genitalia')
            male2Positions = tracks.xs(male2Name, level='individuals', axis=1)
            male2_abdomenlength = getBodypartDistance(male2Positions, 'abdomenTop', 'genitalia')

            mean_abdomenlength_ind1 = np.nanmean(male1_abdomenlength)
            mean_abdomenlength_ind2 = np.nanmean(male2_abdomenlength)
            mean_abdomenlength_ind3 = np.nanmean(female_abdomenlength)

            mean_abdomen_length = [mean_abdomenlength_ind1,mean_abdomenlength_ind2,mean_abdomenlength_ind3]
            indices = np.argsort(mean_abdomen_length)
            indices = indices + 1

            male1Name = 'ind'+str(indices[0])
            male2Name = 'ind'+str(indices[1])
            femaleName = 'ind'+str(indices[2])

            # get female positions
            femalePositions = tracks.xs(femaleName, level='individuals', axis=1)
            femalePositions = removeJumps(femalePositions, maxJump)
            femaleCenter = getAnimalCentroid(femalePositions)

            # get male1 positions
            male1Positions = tracks.xs(male1Name, level='individuals', axis=1)
            male1Positions = removeJumps(male1Positions, maxJump)
            male1Center = getAnimalCentroid(male1Positions)

            # get male2 positions
            male2Positions = tracks.xs(male2Name, level='individuals', axis=1)
            male2Positions = removeJumps(male2Positions, maxJump)
            male2Center = getAnimalCentroid(male2Positions)

            # not detecting copulation here, so used all frames
            # could potentially include the frame number where copulation took place
            copFrame = len(tracks)

            # create parameter DataFrames
            femaleParams = pd.DataFrame(getBodypartAngle(femalePositions, 'thoraxCenter', 'headTop'),
                                        columns=['heading'])
            male1Params = pd.DataFrame(getBodypartAngle(male1Positions, 'thoraxCenter', 'headTop'),
                                       columns=['heading'])
            male2Params = pd.DataFrame(getBodypartAngle(male2Positions, 'thoraxCenter', 'headTop'),
                                       columns=['heading'])

            # get additional female parameters
            femaleParams['XCenter'], femaleParams['YCenter'] = femaleCenter[:copFrame, 0], femaleCenter[:copFrame, 1]
            femaleParams['abdomenLength'] = getBodypartDistance(femalePositions, 'abdomenTop', 'genitalia')
            femaleParams['abdomenWidth'] = getBodypartDistance(femalePositions, 'abdomenLeft', 'abdomenRight')
            femaleParams['linSpeed'] = np.concatenate(
                (np.zeros(1), np.sqrt(np.sum(np.square(np.diff(femaleCenter[:copFrame, ], axis=0)), axis=1))))
            leftW = getBodypartAngle(femalePositions, 'thoraxCenter', 'wingLeft')
            rightW = getBodypartAngle(femalePositions, 'thoraxCenter', 'wingRight')
            femaleParams['leftWingAngle'] = wrapTo2pi(circularDistance(femaleParams['heading'], leftW))
            femaleParams['rightWingAngle'] = wrapTo2pi(circularDistance(femaleParams['heading'], rightW))
            femaleParams['interWingDistance'] = getBodypartDistance(femalePositions, 'wingRight', 'wingLeft')

            # get additional male1 parameters
            male1Params['XCenter'], male1Params['YCenter'] = male1Center[:copFrame, 0], male1Center[:copFrame, 1]
            male1Params['abdomenLength'] = getBodypartDistance(male1Positions, 'abdomenTop', 'genitalia')
            male1Params['abdomenWidth'] = getBodypartDistance(male1Positions, 'abdomenLeft', 'abdomenRight')
            male1Params['linSpeed'] = np.concatenate(
                (np.zeros(1), np.sqrt(np.sum(np.square(np.diff(male1Center[:copFrame, ], axis=0)), axis=1))))
            leftW = getBodypartAngle(male1Positions, 'thoraxCenter', 'wingLeft')
            rightW = getBodypartAngle(male1Positions, 'thoraxCenter', 'wingRight')
            male1Params['leftWingAngle'] = wrapTo2pi(circularDistance(male1Params['heading'].interpolate(), leftW))
            male1Params['rightWingAngle'] = wrapTo2pi(circularDistance(male1Params['heading'].interpolate(), rightW))
            male1Params['interWingDistance'] = getBodypartDistance(male1Positions, 'wingRight', 'wingLeft')

            # get additional male2 parameters
            male2Params['XCenter'], male2Params['YCenter'] = male2Center[:copFrame, 0], male2Center[:copFrame, 1]
            male2Params['abdomenLength'] = getBodypartDistance(male2Positions, 'abdomenTop', 'genitalia')
            male2Params['abdomenWidth'] = getBodypartDistance(male2Positions, 'abdomenLeft', 'abdomenRight')
            male2Params['linSpeed'] = np.concatenate(
                (np.zeros(1), np.sqrt(np.sum(np.square(np.diff(male2Center[:copFrame, ], axis=0)), axis=1))))
            leftW = getBodypartAngle(male2Positions, 'thoraxCenter', 'wingLeft')
            rightW = getBodypartAngle(male2Positions, 'thoraxCenter', 'wingRight')
            male2Params['leftWingAngle'] = wrapTo2pi(circularDistance(male2Params['heading'], leftW))
            male2Params['rightWingAngle'] = wrapTo2pi(circularDistance(male2Params['heading'], rightW))
            male2Params['interWingDistance'] = getBodypartDistance(male2Positions, 'wingRight', 'wingLeft')


            #Prepare JAABA parameters
            chamberSize = 200 # note 20mm, the unit was scaled up 10 by accident, we corrected it in the last step of the pipeline
            positionRange = np.nanmax(femalePositions[scorer]['headTop']['y']) - np.nanmin(femalePositions[scorer]['headTop']['y'])
            scaleFactor = chamberSize/positionRange

            JAABAParams_male1 = pd.DataFrame(male1Params['XCenter'],
                                            columns=['x'])
            JAABAParams_male1['x'] = male1Params['XCenter']
            JAABAParams_male1['y'] = male1Params['YCenter']
            JAABAParams_male1['theta'] = male1Params['heading']
            JAABAParams_male1['a'] = getBodypartDistance(male1Positions, 'headTop', 'genitalia')/4
            JAABAParams_male1['b'] = male1Params['abdomenWidth']
            JAABAParams_male1['nframes'] = len(tracks)
            JAABAParams_male1['firstframe'] = 1
            JAABAParams_male1['endframe'] = len(tracks)
            JAABAParams_male1['off'] = 0
            JAABAParams_male1['id'] = 1
            JAABAParams_male1['x_mm'] = male1Params['XCenter']*scaleFactor
            JAABAParams_male1['y_mm'] = male1Params['YCenter']*scaleFactor
            JAABAParams_male1['theta_mm'] = male1Params['heading']
            JAABAParams_male1['a_mm'] = JAABAParams_male1['a']*scaleFactor
            JAABAParams_male1['b_mm'] = JAABAParams_male1['b']*scaleFactor
            JAABAParams_male1['sex'] = 'm'
            JAABAParams_male1['dt'] = np.append(np.diff(tstamp),1/sampleRate)
            JAABAParams_male1['fps'] = 60
            JAABAParams_male1['timestamps'] = tstamp
            JAABAParams_male1['wing_anglel'] = male1Params['leftWingAngle']
            JAABAParams_male1['wing_angler'] = male1Params['rightWingAngle']
            JAABAParams_male1['xwingl'] = male1Positions[scorer]['wingLeft']['x']
            JAABAParams_male1['ywingl'] = male1Positions[scorer]['wingLeft']['y']
            JAABAParams_male1['xwingr'] = male1Positions[scorer]['wingRight']['x']
            JAABAParams_male1['ywingr'] = male1Positions[scorer]['wingRight']['y']
            JAABAParams_male1['abdlen_mm'] = male1Params['abdomenLength']*scaleFactor

            JAABAParams_male2 = pd.DataFrame(male2Params['XCenter'],
                                             columns=['x'])
            JAABAParams_male2['x'] = male2Params['XCenter']
            JAABAParams_male2['y'] = male2Params['YCenter']
            JAABAParams_male2['theta'] = male2Params['heading']
            JAABAParams_male2['a'] = getBodypartDistance(male2Positions, 'headTop', 'genitalia')/4
            JAABAParams_male2['b'] = male2Params['abdomenWidth']
            JAABAParams_male2['nframes'] = len(tracks)
            JAABAParams_male2['firstframe'] = 1
            JAABAParams_male2['endframe'] = len(tracks)
            JAABAParams_male2['off'] = 0
            JAABAParams_male2['id'] = 2
            JAABAParams_male2['x_mm'] = male2Params['XCenter']*scaleFactor
            JAABAParams_male2['y_mm'] = male2Params['YCenter']*scaleFactor
            JAABAParams_male2['theta_mm'] = male2Params['heading']
            JAABAParams_male2['a_mm'] = JAABAParams_male2['a']*scaleFactor
            JAABAParams_male2['b_mm'] = JAABAParams_male2['b']*scaleFactor
            JAABAParams_male2['sex'] = 'm'
            JAABAParams_male2['dt'] = np.append(np.diff(tstamp),1/sampleRate)
            JAABAParams_male2['fps'] = 60
            JAABAParams_male2['timestamps'] = tstamp
            JAABAParams_male2['wing_anglel'] = male2Params['leftWingAngle']
            JAABAParams_male2['wing_angler'] = male2Params['rightWingAngle']
            JAABAParams_male2['xwingl'] = male2Positions[scorer]['wingLeft']['x']
            JAABAParams_male2['ywingl'] = male2Positions[scorer]['wingLeft']['y']
            JAABAParams_male2['xwingr'] = male2Positions[scorer]['wingRight']['x']
            JAABAParams_male2['ywingr'] = male2Positions[scorer]['wingRight']['y']
            JAABAParams_male2['abdlen_mm'] = male2Params['abdomenLength']*scaleFactor

            JAABAParams_female = pd.DataFrame(femaleParams['XCenter'],
                                              columns=['x'])
            JAABAParams_female['x'] = femaleParams['XCenter']
            JAABAParams_female['y'] = femaleParams['YCenter']
            JAABAParams_female['theta'] = femaleParams['heading']
            JAABAParams_female['a'] = getBodypartDistance(femalePositions, 'headTop', 'genitalia')/4
            JAABAParams_female['b'] = femaleParams['abdomenWidth']
            JAABAParams_female['nframes'] = len(tracks)
            JAABAParams_female['firstframe'] = 1
            JAABAParams_female['endframe'] = len(tracks)
            JAABAParams_female['off'] = 0
            JAABAParams_female['id'] = 3
            JAABAParams_female['x_mm'] = femaleParams['XCenter']*scaleFactor
            JAABAParams_female['y_mm'] = femaleParams['YCenter']*scaleFactor
            JAABAParams_female['theta_mm'] = femaleParams['heading']
            JAABAParams_female['a_mm'] = JAABAParams_female['a']*scaleFactor
            JAABAParams_female['b_mm'] = JAABAParams_female['b']*scaleFactor
            JAABAParams_female['sex'] = 'f'
            JAABAParams_female['dt'] = np.append(np.diff(tstamp),1/sampleRate)
            JAABAParams_female['fps'] = 60
            JAABAParams_female['timestamps'] = tstamp
            JAABAParams_female['wing_anglel'] = femaleParams['leftWingAngle']
            JAABAParams_female['wing_angler'] = femaleParams['rightWingAngle']
            JAABAParams_female['xwingl'] = femalePositions[scorer]['wingLeft']['x']
            JAABAParams_female['ywingl'] = femalePositions[scorer]['wingLeft']['y']
            JAABAParams_female['xwingr'] = femalePositions[scorer]['wingRight']['x']
            JAABAParams_female['ywingr'] = femalePositions[scorer]['wingRight']['y']
            JAABAParams_female['abdlen_mm'] = femaleParams['abdomenLength']*scaleFactor

            #write files
            JAABAParams_female.to_excel(os.path.join(pathname, filename.replace('.h5', '_femaleJAABA.xlsx')))
            JAABAParams_male1.to_excel(os.path.join(pathname, filename.replace('.h5', '_male1JAABA.xlsx')))
            JAABAParams_male2.to_excel(os.path.join(pathname, filename.replace('.h5', '_male2JAABA.xlsx')))

/var/folders/6h/zr_qlv5n4b7dx4xxn6fwrkpm0000gn/T/ipykernel_21564/3742952326.py:20: RuntimeWarning:

Mean of empty slice

/var/folders/6h/zr_qlv5n4b7dx4xxn6fwrkpm0000gn/T/ipykernel_21564/3742952326.py:20: RuntimeWarning:

Mean of empty slice

/var/folders/6h/zr_qlv5n4b7dx4xxn6fwrkpm0000gn/T/ipykernel_21564/3742952326.py:20: RuntimeWarning:

Mean of empty slice

/var/folders/6h/zr_qlv5n4b7dx4xxn6fwrkpm0000gn/T/ipykernel_21564/3742952326.py:20: RuntimeWarning:

Mean of empty slice

/var/folders/6h/zr_qlv5n4b7dx4xxn6fwrkpm0000gn/T/ipykernel_21564/3742952326.py:20: RuntimeWarning:

Mean of empty slice

/var/folders/6h/zr_qlv5n4b7dx4xxn6fwrkpm0000gn/T/ipykernel_21564/3742952326.py:20: RuntimeWarning:

Mean of empty slice

/var/folders/6h/zr_qlv5n4b7dx4xxn6fwrkpm0000gn/T/ipykernel_21564/3742952326.py:20: RuntimeWarning:

Mean of empty slice

/var/folders/6h/zr_qlv5n4b7dx4xxn6fwrkpm0000gn/T/ipykernel_21564/3742952326.py:20: RuntimeWarning:

Mean of empty slice

